### 作業目的: 熟練以Torchtext進行文本資料讀取

本次作業主要會使用[polarity](http://www.cs.cornell.edu/people/pabo/movie-review-data/)的電影評論來進行使用torchtext資料讀取，學員可以在附件的polarity.tsv看到所使用的資料。

Hint: 這次作業同學可以嘗試使用[torchtext.data.TabularDataset](https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset)，可以更簡易讀取資料

### 載入套件

In [ ]:
!pip install torchtext

In [18]:
import torch
import pandas as pd
import numpy as np
import re
from torchtext.legacy import data, datasets

In [19]:
# 探索資料
# 可以發現資料為文本與類別，而類別即為正評與負評
input_data = pd.read_csv('./polarity.tsv', delimiter='\t', header=None, names=['text', 'label'])
input_data

,text,label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,you've got mail works alot better than it dese...,1
3,jaws is a rare film that grabs your attentio...,1
4,moviemaking is a lot like being the general ma...,1
...,...,...
1995,"if anything , "" stigmata "" should be taken as ...",0
1996,"john boorman's "" zardoz "" is a goofy cinematic...",0
1997,the kids in the hall are an acquired taste .it...,0
1998,there was a time when john carpenter was a gre...,0


### 建立Pipeline生成資料

In [22]:
# 建立Field與Dataset
def remove_non_char(x):
    x = ' '.join(x)
    x = re.sub("[^a-zA-Z\"']", ' ', x)
    x = x.split()
    return x

text_field = data.Field(sequential=True, dtype=torch.float64, lower=True, tokenize='spacy', preprocessing=remove_non_char)
label_field = data.Field(sequential=False)
input_data = data.TabularDataset(path='polarity.tsv', 
                                 format='tsv', 
                                 fields=[('text', text_field), ('label', label_field)])

In [23]:
# 取的examples並打亂順序
examples = input_data.examples
np.random.shuffle(examples)

# 以8:2的比例切分examples
split_idx = int(len(examples) * 0.8)
train_ex = examples[:split_idx]
test_ex = examples[int(len(examples)*0.8):]

# 建立training與testing dataset
train_data = data.Dataset(examples=train_ex, fields={'text': text_field, 'label': label_field})
test_data = data.Dataset(examples=test_ex, fields={'text':text_field, 'label':label_field})

train_data[0].label, train_data[0].text

('1',
 ['so',
  'many',
  'students',
  'strive',
  'to',
  'get',
  'into',
  'schools',
  'such',
  'as',
  'harvard',
  'of',
  'princeton',
  'but',
  'they',
  'just',
  'do',
  "n't",
  'get',
  'the',
  'good',
  'grades',
  'so',
  'many',
  'students',
  'would',
  'do',
  'anything',
  'to',
  'reach',
  'their',
  'goal',
  'of',
  'getting',
  'into',
  'this',
  'schools',
  'legend',
  'has',
  'it',
  'that',
  'if',
  'your',
  'roommate',
  'commits',
  'suicide',
  'the',
  'remaining',
  'roommates',
  'will',
  'get',
  'an',
  'automatic',
  'chris',
  'michael',
  'vartan',
  'and',
  'tim',
  'matthew',
  'lillard',
  'are',
  'two',
  'guys',
  'who',
  'really',
  'want',
  'the',
  'to',
  'get',
  'into',
  'harvard',
  'how',
  'far',
  'will',
  'they',
  'go',
  'to',
  'get',
  'their',
  'chris',
  'and',
  'tim',
  'are',
  'also',
  'roommates',
  'with',
  'rand',
  'played',
  'by',
  'randall',
  'batinkoff',
  'the',
  'two',
  'scheme',
  'to',
  

In [24]:
# 建立字典
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

print(f"Vocabularies of index 0-5: {text_field.vocab.itos[:10]} \n")
print(f"words to index {text_field.vocab.stoi}")

Vocabularies of index 0-5: ['<unk>', '<pad>', 'the', 'a', 'and', 'of', 'to', 'is', 'in', "'s"] 

words to index defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f027ce6c520>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, "'s": 9, '"': 10, 'it': 11, 'that': 12, 'as': 13, 'with': 14, 'for': 15, 'his': 16, 'this': 17, 'film': 18, 'he': 19, 'but': 20, 'i': 21, 'on': 22, 'are': 23, 'by': 24, "n't": 25, 'be': 26, 'one': 27, 'movie': 28, 'an': 29, 'not': 30, 'who': 31, 'you': 32, 'was': 33, 'have': 34, 'from': 35, 'at': 36, 'has': 37, 'they': 38, 'her': 39, 'all': 40, 'there': 41, 'like': 42, 'so': 43, 'out': 44, 'about': 45, 'up': 46, 'more': 47, 'what': 48, 'when': 49, 'or': 50, 'which': 51, 'their': 52, 'she': 53, 'do': 54, 'some': 55, 'just': 56, 'does': 57, 'if': 58, 'into': 59, 'we': 60, 'him': 61, 'even': 62, 'good': 63, 'only': 64, 'than': 65, 'no': 66, 'time': 67, "'": 68, 'can': 69, 'would': 70, 'mo

In [25]:
# create iterator for training and testing data
train_iter, test_iter = data.Iterator.splits(datasets=(train_data, test_data),
                                             batch_sizes=(3, 3),
                                             repeat=False,  
                                             sort_key = lambda ex: len(ex.text))

In [26]:
for train_batch in train_iter:
    print(train_batch.text, train_batch.text.shape)
    print(train_batch.label, train_batch.label.shape)
    break

tensor([[2.4440e+03, 8.0000e+01, 1.5670e+03],
        [2.0000e+00, 1.9320e+03, 1.2740e+03],
        [1.1170e+03, 8.8000e+01, 7.5160e+03],
        ...,
        [1.0000e+00, 1.0000e+00, 3.6068e+04],
        [1.0000e+00, 1.0000e+00, 3.5390e+03],
        [1.0000e+00, 1.0000e+00, 2.4606e+04]], dtype=torch.float64) torch.Size([798, 3])
tensor([2, 2, 2]) torch.Size([3])
